## So far:
pad sentence and record ending index

But cannot index properly...

So how can I do batching??? Well, pad it with . zero. Just let it learn? How the effect of 0 will not be too big

In [2]:
from data_pre import data_preprocessing
from data_pre import data_preprocessing
from WAC import WAC
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
import time
from torch.utils.data import DataLoader, TensorDataset

(voc_ix, train_data,test_data, dev_data) = data_preprocessing()


runtime: 1.5681426525115967


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
from WAC_ATT import WAC_ATT


'''
load and prepare data
'''
#(voc_ix, trainX, trainy, testX, testy, devX, devy) = data_preprocessing()
#print("finish preparing data\n")

'''
set parameters
'''
## set hyperparameters
VOCAB_SIZE = len(voc_ix) + 1
EMBEDDING_DIM = 100
n_epoch = 20
batch_size = 500
eval_per = 20000/batch_size
PATH = "../model/wac_batch.pt"

## define model
model = WAC_ATT(EMBEDDING_DIM, VOCAB_SIZE)
optimizer = optim.Adagrad(model.parameters(), lr = 1e-2)

## Experiment with training

In [25]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
dataiter = iter(train_loader)
X,y,lens = dataiter.next()

In [26]:
X.shape

torch.Size([500, 56])

In [27]:
VOCAB_SIZE = len(voc_ix) + 1
embedding_dim = 100

In [28]:
word_embeddings = nn.Embedding(VOCAB_SIZE, embedding_dim, sparse = True)
u = nn.Embedding(1,embedding_dim) 

In [29]:
embeds = word_embeddings(X)
embeds.shape

torch.Size([500, 56, 100])

In [30]:
cosine = nn.CosineSimilarity(dim = 2)

In [31]:
maxlen = X.size(1)
mask = torch.arange(maxlen)[None,:] < lens[:,None]

## compute attention
att = torch.exp(cosine(u.weight.data.view(1,1,-1), embeds))

att[~mask] = float(0) 
att = att/att.sum()

In [34]:
bs, maxlen = X.size()

In [39]:
att.size()

torch.Size([500, 56])

In [38]:
X.size()

torch.Size([500, 56])

In [37]:
mask.size()

torch.Size([500, 56])

In [41]:
torch.mul(mask.float(), att)

tensor([[0.0002, 0.0002, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0002, 0.0002, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)

In [21]:
att.sum(dim = 1, keepdim=True).shape

torch.Size([500, 1])

In [23]:
att /= att.sum(dim = 1, keepdim=True)

In [24]:
att.shape

torch.Size([500, 56])

In [20]:
(att.t()/att.sum(dim = 1)).t()

tensor([[0.0739, 0.0818, 0.0718,  ..., 0.0000, 0.0000, 0.0000],
        [0.1399, 0.1494, 0.1275,  ..., 0.0000, 0.0000, 0.0000],
        [0.0874, 0.0821, 0.0744,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.3660, 0.3153, 0.3187,  ..., 0.0000, 0.0000, 0.0000],
        [0.0283, 0.0260, 0.0356,  ..., 0.0000, 0.0000, 0.0000],
        [0.1749, 0.1519, 0.1619,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<TBackward>)

In [13]:
torch.mul(att.view(att.size()[0],att.size()[1],1), embeds).shape

torch.Size([500, 56, 100])

In [69]:
torch.sum?

In [68]:
att.view(1,att.size()).shape

TypeError: view(): argument 'size' must be tuple of ints, but found element of type torch.Size at pos 2

In [45]:
att.shape

torch.Size([500, 56])

In [41]:
embeds.shape

torch.Size([500, 56, 100])

In [46]:
torch.matmul?

In [9]:

## training
losses = []
accs = []
i = 0
best_dev_acc = 0

myloss = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
start = time.time()
for epoch in range(n_epoch):
    print("epoch " + str(epoch))
    
    #dataloaders
    # make sure to SHUFFLE your data
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    for X,y,lens in train_loader:

        if i % eval_per == 0:
            print("time: {}".format(time.time() - start))
            acc = model.evaluate(dev_data.tensors)
            if acc > best_dev_acc:
                best_dev_acc = acc
                torch.save(model, PATH)
            print("accuracy on dev: " + str(acc))
            accs.append(acc)

        # Step 3. Run our forward pass.
        prob = model.forward(X, lens)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        #loss_sent = - y*log(prob) - (1-y)*log(1-prob)
        loss = myloss(prob, y.float())
        #loss += loss_sent

        loss.backward()
        optimizer.step()
        model.zero_grad()
        i +=1

    losses.append(loss.item())
    runtime = time.time() - start
print("runtime: " + str(runtime) + "s")

model_best = torch.load(PATH)
model_best.eval()
acc_dev = model_best.evaluate(dev_data.tensors)
print("best model acc on dev: " + str(acc_dev))
acc_test = model_best.evaluate(test_data.tensors)
print("best model acc on test: " + str(acc_test))

epoch 0
time: 0.010152816772460938
accuracy on dev: 0.5091743119266054


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


time: 3.1987669467926025
accuracy on dev: 0.6754587155963303
time: 6.3477160930633545
accuracy on dev: 0.7213302752293578
time: 9.533467769622803
accuracy on dev: 0.7649082568807339


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([349])) that is different to the input size (torch.Size([349, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


epoch 1
time: 12.702982902526855
accuracy on dev: 0.7786697247706422
time: 15.80125904083252
accuracy on dev: 0.783256880733945
time: 19.02666687965393
accuracy on dev: 0.7935779816513762
epoch 2
time: 22.10341191291809
accuracy on dev: 0.7947247706422018
time: 25.053172826766968
accuracy on dev: 0.8004587155963303
time: 28.016375064849854
accuracy on dev: 0.801605504587156
time: 30.925309896469116
accuracy on dev: 0.8084862385321101
epoch 3
time: 33.86204385757446
accuracy on dev: 0.8027522935779816
time: 36.84261202812195
accuracy on dev: 0.805045871559633
time: 39.788776874542236
accuracy on dev: 0.8073394495412844
epoch 4
time: 42.9779748916626
accuracy on dev: 0.8073394495412844
time: 46.166455030441284
accuracy on dev: 0.8142201834862385
time: 49.41793894767761
accuracy on dev: 0.8119266055045872
epoch 5
time: 52.51468896865845
accuracy on dev: 0.8073394495412844
time: 55.61316990852356
accuracy on dev: 0.8084862385321101
time: 58.71389889717102
accuracy on dev: 0.809633027522935

KeyboardInterrupt: 

In [64]:
# obtain one batch of training data
batch_size = 500
train_loader = DataLoader(train_data, shuffle=F, batch_size=batch_size)

dataiter = iter(train_loader)
X, y,lens = dataiter.next()

embeds = model.word_embeddings(X)
embeds.shape

torch.Size([500, 56, 100])

In [65]:
X.shape

torch.Size([500, 56])

In [72]:
maxlen = X.size(1)
mask = torch.arange(maxlen)[None,:] < lens[:,None]
embeds[~mask] = float(0)

In [75]:
X[0,:]

tensor([12407, 18639,  9777, 19227, 11647, 10646,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0])

In [76]:
(embeds.sum(dim = 1).t()/lens.float()).t()[0,:]/model.word_embeddings(X[0,:6]).mean(dim = 0)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<DivBackward0>)

In [62]:
model.word_embeddings(X[0,:9]).shape

torch.Size([9, 100])

In [71]:
lens

tensor([ 6,  1,  4,  3, 10,  9,  3, 20,  8, 21, 10, 12,  7, 13,  6, 13,  9, 12,
         6, 13,  5,  2,  6,  3,  2, 11, 20, 15, 25,  3, 13,  4,  6, 10,  2,  3,
         3, 27,  5, 19,  9, 18,  3,  5,  1,  9, 19, 15,  6, 14,  4,  6, 11, 18,
         9,  6, 14,  8,  5, 22, 10, 15, 10,  4,  8, 13,  4, 17,  7,  6,  1, 23,
         9, 27,  2,  7,  5,  1,  5,  4, 12,  2, 27, 23, 31,  3, 25,  8,  7,  4,
         6,  5, 19,  8,  4,  1,  6, 14,  5,  2,  3, 15,  4,  9, 11,  1,  2,  9,
         1,  8, 30, 14,  3, 15, 15,  2,  4, 16, 11,  4,  4,  2, 13, 14, 12,  8,
        12,  2,  2, 19, 12,  6, 10, 19,  6, 16,  7,  3,  7, 34, 10,  6,  8,  2,
         4,  5,  1, 18, 16,  5, 18, 10,  8,  6, 22,  1,  7,  2,  2,  8,  2,  2,
         7,  1,  6,  6,  4, 12, 33, 17,  7,  2,  1, 20,  8,  5,  1,  8,  2,  5,
         5,  1, 34,  3, 11,  1,  7,  3, 10,  2,  5, 30,  8,  9,  2, 25, 11, 18,
         4, 11,  4,  3,  3,  9,  8, 10,  6,  3,  3,  2,  4,  8,  3, 33, 15,  3,
         3,  7, 13, 10,  9,  3, 28,  3, 

In [56]:
maxlen/lens

tensor([ 6,  8, 14,  9, 28,  9,  2,  9,  5, 18, 28,  2, 56, 14,  6, 56,  9,  3,
         4, 56,  4,  2, 56, 11, 56,  2,  3,  8,  5,  7, 11,  4,  4,  4,  8,  5,
         8,  8, 18,  8,  8,  5, 28,  3, 11,  4,  4,  4, 28, 14,  4, 18, 14, 28,
         9, 18,  2,  2,  1,  9,  3, 28, 18, 56, 11,  5, 56,  7, 14,  8,  6, 56,
        56, 28,  4,  7,  2, 14, 14, 28,  4,  4, 18,  2, 56,  5, 28,  3,  2,  9,
         4, 28,  2,  3,  3,  3,  2,  2,  9, 11,  4, 14, 18, 11, 28, 14,  4, 18,
         2, 14,  3, 18, 18, 28, 11, 56,  5,  4,  5,  4,  2, 14,  9, 28, 56,  2,
        14,  8,  7,  7, 14, 28,  4,  9,  2, 14, 56,  5,  7,  9, 14,  2, 18,  4,
        56,  2,  5, 11, 18,  4, 11,  4,  5,  3,  9, 11,  9,  7,  3,  5, 28,  2,
        56, 56,  5, 28,  8,  5, 56,  1, 56, 56, 18, 18,  3,  4, 28,  4,  7,  3,
         8,  9,  4, 28,  6,  8,  4,  6,  2, 28, 56, 18,  1,  7,  8, 56,  3,  2,
         6,  6,  6, 56, 18, 56, 28,  2,  7,  6, 11,  7, 28, 14,  7,  6, 56, 14,
         2, 28,  6,  4,  3,  2,  9,  1, 

In [22]:
embeds.shape

torch.Size([500, 56, 100])

In [27]:
embeds[2,6,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], grad_fn=<SliceBackward>)

In [44]:
(embeds.mean(dim = 1).t()*(maxlen/lens.float())).t()[0,:].shape

torch.Size([100])

In [38]:
torch.matmul(maxlen/lens.float(),embeds.mean(dim = 1)).shape

torch.Size([100])

In [5]:
model.predict(X)

RuntimeError: t() expects a 2D tensor, but self is 3D

In [6]:
model.evaluate(dev_data.tensors)

0.5355504587155964

In [16]:
dev_data.tensors[1].shape

torch.Size([872])

In [27]:
x = dev_data.tensors[0][0,:]
y = dev_data.tensors[1][0]

model.forward(x.unsqueeze(0),lens = torch.Tensor(1))

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'other'

In [35]:
model.forward(x.unsqueeze(0),lens = torch.tensor([1], dtype = torch.long))

tensor([[0.4732]], grad_fn=<SigmoidBackward>)

In [ ]:
dtype=torch.long, device=device)